In [ ]:
# -*- coding: utf-8 -*-
#Use to import pandas
import pandas as pd
import numpy as np
import math
from scipy import stats
from scipy.stats import norm
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import itertools
import sympy as sp
from sympy import *
from __future__ import division
sp.init_printing()
import tkinter as tk
from tkinter import filedialog
import colorama


#Através de uma lista de varáveis explicativas retorna a regressão linear multipla
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 100, bg = 'lightsteelblue')
canvas1.pack()

def getcsv ():
    global df1
    root.update()
    import_file_path = filedialog.askopenfilename()
    root.destroy()
    df1 = pd.read_csv (import_file_path, decimal=",",encoding='latin1')
    
browseButton_csv = tk.Button(text='Import csv File', command=getcsv, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 50, window=browseButton_csv)

root.mainloop()

df1=df1.drop(columns=['Dado'])

av=df1.iloc[-1,:] #dados do avaliando
av=av.iloc[:-1,] #dados do avaliando retirando o valor nulo

df2=df1.iloc[:-1,:] #Retirando a linha dos dados do avaliando

nc,nl=df2.shape

def rglmv(df2,av):

    dm=df2.T
    nc,nl=df2.shape

    y=df2.iloc[:,-1] #Vetor dos valores totais 'ou unitários' 
    yt=y.T

    vet=np.ones((nc,), dtype=int) #Vetor unitário a ser inserido na primeira coluna da Matriz Ddos x Variaveis=x
    vet=(pd.DataFrame(vet)).T

    xt=pd.concat([vet, dm.iloc[:-1,:]])

    xt=xt.values 
    x=xt.T #Vetor das observações das variáveis explicativas

    ci=np.matmul(xt,x)

    c=np.linalg.inv(ci)
    xty=np.matmul(xt,y)

    b=np.matmul(c,xty) #Vetor dos coeficientes regressores
    bt=b.T

    yreg=np.matmul(b,xt) #Vetor das estimativas

    ssy=np.matmul(yt,y) # Soma dos quadrados do valores
    sse=ssy-np.matmul(np.matmul(bt,xt),y) #A soma dos quadrados do erros será
    sso = (nc)*((np.mean(y))**2)# Soma dos quadrados do valores
    sst = ssy-sso #Soma dos quadrados totais
    ssr = sst-sse
    r2=ssr/sst

    gle=nc-nl

    Se=math.sqrt(abs(sse/gle))

    dpb=[abs(b[i]/(Se*math.sqrt(c[i][i]))) for i in range(nl)] # Valores relativos dos erros dos regressores' - Teste t

    sig=[(1-stats.t.cdf(dpb[i],df=gle))*2 for i in range(nl)] # Valores das significâncias dos testes t Bicaudal

    psig=max(sig)

    msr=ssr/(nl-1) #Erro médio da regressão
    mse=sse/gle #Erro médio
    mst=sst/(nc-1) #Erro médio total
    F=msr/mse

    sigF=1-stats.f.cdf(F,dfn=nc+1,dfd=gle)

    av=av.values
    av = np.append([1],av,axis=0)
    vav=np.matmul(b,av) #Valor estimado do imóvel

    avc=np.matmul(av,c)
    Ve=np.matmul(avc,av)*(Se**2) # Variancia da estimativa

    t6=stats.t.ppf(0.9,df=gle) #Valor do p-valor para o qual a probabilidade seja menor que 80% bicaudal.
    sic=t6*math.sqrt(Ve) #Metade do intervalo de confiança

    vmin=vav-sic
    vmax=vav+sic

    ampic=2*sic/vav

    return (r2,psig,ampic,b,nc,nl,gle,Se,sig,sigF,F,vav,vmin,vmax,t6,Ve,y,yreg,x,c,xt,mse,sse,sst)

#....................................#Mudança de Variável

exc=np.where(df2<=0)
excl=[*exc[1]]
rng=[*range(nl)]

for i in excl:
 rng.remove(i); #Apenas as colunas que apresentam valores positivos

mod=[] #Vetores que representam cada modelo
if (nl)<=5:
    for i in itertools.product(range(7), repeat=nl):
         mod.append(i);
else:
    for i in range((7**5)*(nl)):
         mod=np.zeros(nl).tolist()
         mod.append(randint(0,6));
    mod = np.reshape(mod, (nl, 7**5)).T
    mod=mod.tolist();

qtdmod=[*range(len(mod))] #Excluindo operações de mudança de variávei em numeros <= zero
modd=[]
for i in range(7**(nl)):
 for j in excl:
  if mod[i][j]==1 or mod[i][j]==2 or mod[i][j]==5 or mod[i][j]==6:
    qtdmod.remove(i);
    
modf=[]
for i in qtdmod:
 modf.append(mod[i]);

#Mudança de variáveis

var_dict={}

for k in range(len(modf)):
 df4=df2.copy()
 for i in range(nl):
  if modf[k][i]==0:
   df4.iloc[:,i]=df2.iloc[:,i].copy()
  if modf[k][i]==1:
   df4.iloc[:,i]=1/df2.iloc[:,i].copy()
  if modf[k][i]==2:
   df4.iloc[:,i]=np.log(df2.iloc[:,i].copy())
  if modf[k][i]==3:
   df4.iloc[:,i]=(df2.iloc[:,i].copy())**2
  if modf[k][i]==4:
   df4.iloc[:,i]=np.sqrt(df2.iloc[:,i].copy())
  if modf[k][i]==5:
   df4.iloc[:,i]=1/((df2.iloc[:,i].copy())**2)
  if modf[k][i]==6:
   df4.iloc[:,i]=1/np.sqrt(df2.iloc[:,i].copy());
  var_dict[k] = df4;

av_dict={}

for k in range(len(modf)):
 df5=av.copy()
 for i in range(nl-1):
  if modf[k][i]==0:
   df5.iloc[:i,]=av.iloc[:i,].copy()
  if modf[k][i]==1:
   df5.iloc[:i,]=1/av.iloc[:i,].copy()
  if modf[k][i]==2:
   df5.iloc[:i,]=np.log(av.iloc[:i,].copy())
  if modf[k][i]==3:
   df5.iloc[:i,]=(av.iloc[:i,].copy())**2
  if modf[k][i]==4:
   df5.iloc[:i,]=np.sqrt(av.iloc[:i,].copy())
  if modf[k][i]==5:
   df5.iloc[:i,]=1/((av.iloc[:i,].copy())**2)
  if modf[k][i]==6:
   df5.iloc[:i,]=1/np.sqrt(av.iloc[:i,].copy());
  av_dict[k] = df5;

#...................................#

rstd=rglmv(df2,av) #regressão linear sem mudança de variáveis modelo zero
nc=rstd[4]
nl=rstd[5]
gle=rstd[6]
yd=rstd[16]
sst=rstd[23]

rstd_dict={}

for k in range(len(modf)):
 rstd_dict[k]=rglmv(var_dict[k],av_dict[k]);

sigt_list=[]
yest_list=[]
filtroyneg=[] #Filtrar e evitar erros com yreg negativo de valor de imóvel
for k in range(len(modf)):
 if modf[k][nl-1]==0: #Desfazendo a troca de variaveis para achar o valor estimado dos dados
  yest_list.append(rstd_dict[k][17]);
  filtroyneg.append(k)
 if modf[k][nl-1]==1:
  yest_list.append(1/rstd_dict[k][17])
  filtroyneg.append(k)
 if modf[k][nl-1]==2 and np.all(rstd_dict[k][17]>0):
  yest_list.append(np.exp(rstd_dict[k][17]))
  filtroyneg.append(k)
 if modf[k][nl-1]==3 and np.all(rstd_dict[k][17]>0):
  yest_list.append(np.sqrt(rstd_dict[k][17]))
  filtroyneg.append(k)
 if modf[k][nl-1]==4:
  yest_list.append(rstd_dict[k][17]**2)
  filtroyneg.append(k)
 if modf[k][nl-1]==5 and np.all(rstd_dict[k][17]>0):
  yest_list.append(1/np.sqrt(rstd_dict[k][17]))
  filtroyneg.append(k)
 if modf[k][nl-1]==6:
  yest_list.append(1/(rstd_dict[k][17]**2))
  filtroyneg.append(k);

Eest=[]
for k in range(len(yest_list)):
 Eest.append(yest_list[k]-yd.values);

SQE_l=[]
for k in range(len(yest_list)):
     SQE_l.append(sum(Eest[k]*Eest[k]));
        
R2E_l=[]
for k in range(len(yest_list)):
     R2E_l.append(1-SQE_l[k]/sst);
        
R2E_l_ord=sorted(R2E_l, reverse=True)

NtR2E_l=[] #Classificacao das determinacoes nos modelos
for k in R2E_l:
 NtR2E_l.append(R2E_l_ord.index(k));

VavE_l=[]
filtrovav=[]

for k in filtroyneg:
 if modf[k][nl-1]==0:
  VavE_l.append(rstd_dict[k][11])
  filtrovav.append(k)
 if modf[k][nl-1]==1:
  VavE_l.append(1/rstd_dict[k][11])
  filtrovav.append(k)
 if modf[k][nl-1]==2 and rstd_dict[k][11]<40:
  VavE_l.append(np.exp(rstd_dict[k][11]))
  filtrovav.append(k)
 if modf[k][nl-1]==3:
  VavE_l.append(np.sqrt(rstd_dict[k][11]))
  filtrovav.append(k)
 if modf[k][nl-1]==4:
  VavE_l.append(rstd_dict[k][11]**2)
  filtrovav.append(k)
 if modf[k][nl-1]==5:
  VavE_l.append(1/np.sqrt(rstd_dict[k][11]))
  filtrovav.append(k)
 if modf[k][nl-1]==6:
  VavE_l.append(1/(rstd_dict[k][11]**2));
  filtrovav.append(k);

VEMin_l=[]
VEMax_l=[]
filtroampl=[]

for k in filtrovav:
 if modf[k][nl-1]==0:
  VEMin_l.append(rstd_dict[k][12])
  VEMax_l.append(rstd_dict[k][13])
  filtroampl.append(k)
 if modf[k][nl-1]==1:
  VEMin_l.append(1/rstd_dict[k][12])
  VEMax_l.append(1/rstd_dict[k][13])
  filtroampl.append(k)
 if modf[k][nl-1]==2 and rstd_dict[k][13]<40:
  VEMin_l.append(np.exp(rstd_dict[k][12]))
  VEMax_l.append(np.exp(rstd_dict[k][13]))
  filtroampl.append(k)
 if modf[k][nl-1]==3:
  VEMin_l.append(np.sqrt(rstd_dict[k][12]))
  VEMax_l.append(np.sqrt(rstd_dict[k][13]))
  filtroampl.append(k)
 if modf[k][nl-1]==4:
  VEMin_l.append(rstd_dict[k][12]**2)
  VEMax_l.append(rstd_dict[k][13]**2)
  filtroampl.append(k)
 if modf[k][nl-1]==5:
  VEMin_l.append(1/np.sqrt(rstd_dict[k][12]))
  VEMax_l.append(1/np.sqrt(rstd_dict[k][13]))
  filtroampl.append(k)
 if modf[k][nl-1]==6:
  VEMin_l.append(1/(rstd_dict[k][12]**2))
  VEMax_l.append(1/(rstd_dict[k][13]**2))
  filtroampl.append(k);

ampl_l=[]
filtroampl2=[]
for k in range(len(filtroampl)):
 if VavE_l[k]>0:
  ampl_l.append(abs(VEMax_l[k]-VEMin_l[k])/VavE_l[k])
  filtroampl2.append(filtroampl[k]);
    
ampl_l_ord=sorted(ampl_l)

Ntampl_l=[] #Classificacao das amplitudes nos modelos
for k in ampl_l:
 Ntampl_l.append(ampl_l_ord.index(k));

sigt_list=[]
for k in filtroampl2:
 sigt_list.append(rstd_dict[k][1]);

sigt_l_ord=sorted(sigt_list)


Ntsigt_l=[] #Classificacao das amplitudes nos modelos
for k in sigt_list:
 Ntsigt_l.append(sigt_l_ord.index(k));

Nota=[] #combinacao da melhor determinacao, amplitude e significancia
for k in filtroampl2:
 Nota.append(Ntsigt_l[filtroampl2.index(k)]+Ntampl_l[filtroampl2.index(k)]+NtR2E_l[filtroyneg.index(k)]);

Nota_ord=sorted(Nota)

modmelnota=filtroampl2[Nota.index(Nota_ord[0])] #Modelo com a melhor nota - combinacao de R2,Sigfc e Ampl

Tab_fund=[] #Valor dos itens de fundamentacao 
for k in Nota_ord:
    Tab_fund.append(k)
    Tab_fund.append(filtroampl2[Nota.index(k)])
    Tab_fund.append(R2E_l[filtroyneg.index(filtroampl2[Nota.index(k)])])
    Tab_fund.append(NtR2E_l[filtroyneg.index(filtroampl2[Nota.index(k)])])
    Tab_fund.append(sigt_list[Nota.index(k)])
    Tab_fund.append(Ntsigt_l[Nota.index(k)])
    Tab_fund.append(ampl_l[Nota.index(k)])
    Tab_fund.append(Ntampl_l[Nota.index(k)]);

Tab_fund = np.reshape(Tab_fund , (len(Nota_ord), 8))

Tab_fund=pd.DataFrame(Tab_fund)

Tab_fund.columns = ['Nota da Combinação','Nº do Modelo','R²',
                     'Classificação do R²','Significância','Classificação da Sign.',
                      'Amplitude do IC','Classificação da Ampl. do IC.']

Tab_fund['Amplitude do IC'] = pd.Series(["{0:.2f}%".format(val*100) for val in Tab_fund['Amplitude do IC']])
Tab_fund['Significância'] = pd.Series(["{0:.2f}%".format(val*100) for val in Tab_fund['Significância']])
Tab_fund['R²'] = pd.Series(["{0:.3f}".format(val) for val in Tab_fund['R²']])
print ('\033[1m'+'\033[91m'+"Os Modelos com melhores combinações de determinação,Siginificância e Amplitude")
display(Tab_fund.head(10))

print ('\033[1m'+'\033[91m'+"Os Modelos com melhores R²")
Tab_fund.sort_values('Classificação do R²', inplace = True)
display(Tab_fund.head(10))

print ('\033[1m'+'\033[91m'+"Os Modelso com melhores Significância")
Tab_fund.sort_values('Classificação da Sign.', inplace = True)
display(Tab_fund.head(10))

print ('\033[1m'+'\033[91m'+"Oa Modeloa com menor amplitude de Intervalo de Confiança")
Tab_fund.sort_values('Classificação da Ampl. do IC.', inplace = True)
display(Tab_fund.head(10))

modesc = int(input("Escolha na segunda coluna das tabelas acima o modelo que considerar mais adequado"+
                   " escrevendo aqui o 'Nºdo Modelo': "));

R2E=R2E_l[filtroyneg.index(modesc)]
VavE=VavE_l[filtrovav.index(modesc)]
VEMin=VEMin_l[filtroampl.index(modesc)]
VEMax=VEMax_l[filtroampl.index(modesc)]
ampl=ampl_l[filtroampl2.index(modesc)]
sigt=sigt_list[filtroampl2.index(modesc)]
yest=yest_list[filtroyneg.index(modesc)]

r2=rstd_dict[modesc][0]
psig=rstd_dict[modesc][1]
b=rstd_dict[modesc][3]
Se=rstd_dict[modesc][7]
sig=rstd_dict[modesc][8]
sigF=rstd_dict[modesc][9]
F=rstd_dict[modesc][10]
vav=rstd_dict[modesc][11]
vmin=rstd_dict[modesc][12]
vmax=rstd_dict[modesc][13]
t6=rstd_dict[modesc][14]
Ve=rstd_dict[modesc][15]
y=rstd_dict[modesc][16]
yreg=rstd_dict[modesc][17]
x=rstd_dict[modesc][18]
c=rstd_dict[modesc][19]
xt=rstd_dict[modesc][20]
mse=rstd_dict[modesc][21]
sse=rstd_dict[modesc][22]

print("O Vetor dos coeficientes regressores é",b)

t = [ '*x'+str(i) for i in range(nl)]

z=[]
# Função Condicinal para inserir sinal em elementos da string 
def adcsinal(item): 
    if int(item) >0: 
        return '+'+str(round(item,4))
    return str(round(item,4))
      
z = [adcsinal(item) for item in b]
print ('\033[0m')

y_eqest=[]
if modf[modesc][nl-1]==0:
    y_eqest.append('y=')
if modf[modesc][nl-1]==1:
    y_eqest.append('(1/y=)')
if modf[modesc][nl-1]==2:
    y_eqest.append('exp(y)=')
if modf[modesc][nl-1]==3:
    y_eqest.append('sqrt(y)=')
if modf[modesc][nl-1]==4:
    y_eqest.append('y**2=')
if modf[modesc][nl-1]==5:
    y_eqest.append('1/sqrt(y)=')
if modf[modesc][nl-1]==6:
    y_eqest.append('(1/y**2)=');

t_est=['']
for k in range(nl-1):
    if modf[modesc][k]==0:
        t_est.append('*x'+str(k+1))
    if modf[modesc][k]==1:
        t_est.append('*(1/x'+str(k+1)+')')
    if modf[modesc][k]==2:
        t_est.append('*exp(x'+str(k+1)+')')
    if modf[modesc][k]==3:
        t_est.append('*sqrt(x'+str(k+1)+')')
    if modf[modesc][k]==4:
        t_est.append('*(x'+str(k+1)+'**2)')
    if modf[modesc][k]==5:
        t_est.append('*1/sqrt(x'+str(k+1)+')')
    if modf[modesc][k]==6:
        t_est.append('*1/x'+str(k+1)+'**2');
        
zs=[]
zs = [adcsinal(item) for item in b]

for i in range(nl) :
  zs[i]=zs[i]+t_est[i]; # Concatenando os coeficientes com as variaveis

k_est = "".join(zs)

m_est=str(y_eqest[0])+k_est

eqe = Eq(*map(S, m_est.split('=', 1)))

print ('\033[1m'+'\033[91m'+"A equação da estimativa será")
display(eqe)

if R2E < 0.6:
 r2Ev=', valor muito baixo'
if 0.6<= R2E < 0.8:
 r2Ev=', valor aceitável'
if 0.8<= R2E < 0.9:
 r2Ev=', valor Bom'
else:
 r2Ev=', valor Ótimo'

print ("O coeficiente de determinação da Estimativa é R²=",round(R2E,3),r2Ev)
print ('\033[0m')

if r2 < 0.6:
 r2v=', valor muito baixo'
if 0.6<= r2 < 0.8:
 r2v=', valor aceitável'
if 0.8<= r2 < 0.9:
 r2v=', valor Bom'
else:
 r2v=', valor Ótimo'

print ("O coeficiente de determinação da regressão é R²=",round(r2,3),r2v)
print ('\033[0m')

print (gle,"Graus de Liberdade do Erro")
print("")

print (nl-1,"Graus de Liberdade do Regressão")
print("")

print ("O coeficiente de determinação da regressão ajustado é Ra²=",round(1-(1-R2E)*(nc-1)/gle,3))
print ('\033[1m')
print ("O coeficiente de determinação da regressão ajustado é Ra²=",round(1-(1-r2)*(nc-1)/gle,3))
print ('\033[1m')

r = math.sqrt(R2E)

if r == 0:
 rv=', correlação nula'
if r > 0 and r<0.4:
 rv=', correlação fraca'
if r >= 0.4 and r<0.7:
 rv=', correlação Média'
if r >= 0.7 and r<0.9:
 rv=', correlação Forte'
if r >= 0.9 and r<1:
 rv=', correlação Fortíssima'
else:
 rv=', correlação perfeita';

print ("O coeficiente de correlação da regressão é R=",round(r,3),rv)
print("")


print ("O desvio padrão da regressão é Se=",round(Se,4))
print("")

print ('\033[91m'+"TESTE DE HIPÓTESE DOS REGRESSORES"'\033[91m')

sg=[(str(round(sig[i]*100,3))+"%") for i in range(nl)]

print ("As significâncias dos regressores são, respectivamente",sg)

sgf=[ i for i in range(nl) if sig[i]<0.05]

vsig=[ (df2.columns[i-1]) for i in sgf]

if len(vsig)==len(sig):
 t0="Todos os regressores são significante"
 t1="";
if len(vsig)==1:
 t0="Apenas o regressor de:"
 t1='é significante'
else:
 t0="Apenas os regressores de:"
 t1='são significantes';

print ('\033[1m')

if sgf[0]==0:
 vsig[0]="constante";

vsig = ", ".join(vsig)
print(t0,vsig,t1)

ppsig=0
for i in range(0, nl, 1) :
    if sig[i]==psig:
        ppsig=i;
        ;
if ppsig==0:
 t3="constante"
else:
 t3="variável "+df2.columns[ppsig-1]
;
    
t4 = "A pior siginficância do modelo é %s%%" % round(psig*100,3)

    
print("")
print(t4,"do regressor da",t3)

print("")

print ('\033[91m'+"TESTE DE HIPÓTESE DA REGRESSÃO"'\033[91m')

if sigF<0.01:
 t5="com significância boa e menor que 1%"
else:
 t5="com significância ruim e maior que 1%"
;

print ('\033[0m')
print("O valor de F de Fisher-Snedecor é F=",round(F,3),t5,",Significância =",sigF)



print ('\033[1m')
print("O valor estimado do imóvel pela estimativa é R$",'{:,.2f}'.format(float(VavE)))

print ('\033[91m')
print ("VALORES MÍNIMOS E MÁXIMO NO INTERVALO DE CONFIANÇA DE 80%")

print ('\033[0m')

print("O valor mínimo estimado do imóvel pela regressão é R$",'{:,.2f}'.format(float(VEMin)))
print("O valor máximo estimado do imóvel pela regressão é R$",'{:,.2f}'.format(float(VEMax)))

print("A amplitude do intervalo de confiânça da estimativa é %s%% " % round(100*ampl,2))

print ('\033[91m','\033[1m')
print ("VALORES MÍNIMOS E MÁXIMO NO INTERVALO DE PREDIÇÃO DE 80%")

sip=t6*math.sqrt(Ve+Se**2)
vminp=vav-sip
vmaxp=vav+sip

if modf[modesc][nl-1]==0:
 vminpE=vminp
 vmaxpE=vmaxp
if modf[modesc][nl-1]==1:
 vminpE=(1/vminp)
 vmaxpE=(1/vmaxp)
if modf[modesc][nl-1]==2 and vmaxp<40:
 vminpE=(np.exp(vminp))
 vmaxpE=(np.exp(vmaxp))
if modf[modesc][nl-1]==3:
 vminpE=(np.sqrt(vminp))
 vmaxpE=(np.sqrt(vmaxp))
if modf[modesc][nl-1]==4:
 vminpE=(vminp**2)
 vmaxpE=(vmaxp**2)
if modf[modesc][nl-1]==5:
 vminpE=(1/np.sqrt(vminp))
 vmaxpE=(1/np.sqrt(vmaxp))
if modf[modesc][nl-1]==6:
 vminpE=(1/(vminp**2))
 vmaxpE=(1/(vmaxp**2));
    
ampip=abs((vmaxpE-vminpE))/VavE

if vmaxpE>vminpE:
 vmaxpE=vmaxpE
 vminpE=vminpE
else:
 vmaxpE=vminpE
 vminpE=vmaxpE;
    
print ('\033[0m')
print("A amplitude do intervalo de predição é %s%% "% round(100*ampip,2))
print("O valor mínimo do intervalo de predição é R$",'{:,.2f}'.format(float(vminpE)))
print("O valor máximodo do intervalo de predição é R$",'{:,.2f}'.format(float(vmaxpE)))

print ('\033[91m','\033[1m')
print ("VALORES MÍNIMOS E MÁXIMO NO INTERVALO DE ARBÍTRIO DE 15%")
print ('\033[0m')
print("O valor mínimo do campo de arbítrio é R$",'{:,.2f}'.format(float(VavE*0.85)))
print("O valor máximodo do campo de arbítrio é R$",'{:,.2f}'.format(float(VavE*1.15)))

print ('\033[91m','\033[1m')
print ("ANÁLISE DE RESÍDUOS E OUTLIERS")
print ('\033[0m')

res=y.values-yreg #Resíduo
rrel=res/y #Resíduo relativo aos dados

rrelp=[(str(round(rrel[i]*100,3))+"%") for i in range(nc)]

print ("O resíduos relativos serão, respectivamente",rrelp)

rdp=res/Se #Resíduo em função do desvio da regressão - Resíduo padronizado

print ('\033[1m')

print ("O resíduos em relação ao desvio da regressão serão, respectivamente:")

print (np.round(rdp,3))

otl=[ i+1 for i in range(nl) if rdp[i]>2 or rdp[i]<-2]

plt.rcParams['figure.figsize'] = (11,7)
        
for xn, yn in zip(yreg, rdp):
    color = 'blue'  # non-outlier color
    if not -2<= yn <= 2:  # condition for being an outlier
        color = 'red'  # outlier color
    plt.scatter(xn, yn, color=color)
plt.plot(yreg,2*np.ones((nc), dtype=int),'r-')
plt.plot(yreg,-2*np.ones((nc), dtype=int),'r-')
plt.rcParams['figure.figsize'] = (11,7)
plt.xlabel('Valores Estimados')
plt.ylabel('Erro Padrão')
plt.title('Homocedasticidade')
plt.show()

print ('\033[1m')
print ("Ao total temos",len(otl),"outliers")

if len(otl)==0:
 print ('')
else:
 print ('Os dados',otl,'são outliers');

Hl=np.matmul(x,np.matmul(c,xt)) #Matriz Chapeu para identificação dos pontos de alavanca, o posto da martiz = p+1, logo observa-se o valor acima do dobro do média como possivel influenciante.

dck=[ ((((res[i])**2)*Hl[i,i])/((nl)*mse*((1-Hl[i,i])**2))) for i in range(nc)] #Distância de Cook para identificação de dados influenciantes

for xc, yc in zip(np.linspace(0,nc-1,nc),dck):
    color = 'blue'  # non-outlier color
    if not yc <= 1:  # condition for being an outlier
        color = 'red'  # outlier color
    plt.scatter(xc, yc, color=color)
plt.plot(np.linspace(0,nc-1,nc),np.ones((nc), dtype=int), 'r--')
plt.xlabel('Dados')
plt.ylabel('Distância de Cook')
plt.title('Análise de dados influenciantes')
plt.show()

print ('\033[91m','\033[1m')
print ("TESTE PARA O CASO DE UTILIZAÇÃO DE SÉRIES TEMPORAIS")

dw=[ ((res[i+1]-res[i])**2) for i in range(nc-1)]

tdw=sum(dw)/sse

dwdf = pd.read_csv ('DW.csv',decimal=",",encoding='latin1') #Importando dados do teste de Durbin-Watson
dwT=dwdf.loc[(dwdf['T'])==nc]
dwK=dwT.loc[(dwT['K'])==nl]
dl=(dwK.loc[:'dl']).iloc[0,2]
dh=(dwK.loc[:'dl']).iloc[0,3]

print ('Os teste de Durbin-Watson teve os seguintes valores: dw=',tdw,',dl=',dl,'e dh=',dh)

if 0<= tdw < dl:
 rdw='Como 0<= tdw < dl: autocorrelação positiva - Há dependência entre os resultados temporais'
if dl<= tdw < dh:
 rdw='Como dl<= tdw < dh: Teste Inconclusivo-"Grey Zone"-para dependência entre os resultados temporais'
if dh<= tdw <(4-dh):
 rdw='Como dh<= tdw <(4-dh):Não há autocorrelação de dependência entre os resultados temporais'
if (4-dh)<= tdw <(4-dl):
 rdw='Como (4-dh)<= tdw <(4-dl): Teste Inconclusivo-"Grey Zone"-para dependência entre os resultados temporais'
else:
 rdw='Como (4-dl)<= tdw <4: Indicador de significativa autocorrelação negativa';
print ('\033[0m')
print (rdw)

print ('\033[91m','\033[1m')
print ("ANÁLISE DE ADERÊNCIA")

#Gráfico dos valores observados x estimados
plt.plot(yd,yd,'-y')
scatter_plot = plt.scatter(yd,yest,alpha=0.5)

from sklearn import linear_model
model = linear_model.LinearRegression().fit(yd.values.reshape(-1, 1),yest)
plt.plot(yd.values.reshape(-1, 1), model.predict(yd.values.reshape(-1, 1)), color='blue', linewidth=3)
plt.xticks(())
plt.yticks(())

plt.xlabel('Valores Observados')
plt.ylabel('Valores Estimados')
plt.title('Observados x Estimados')
plt.rcParams['figure.figsize'] = (11,7)
plt.show()


# Histograma dos residuos padronizados
mu = np.mean(rdp) # média dos erros
sigma = np.std(rdp) # Desvio padrão dos erros
xh = rdp

num_bins = 3

n, bins, patches = plt.hist(xh, num_bins, density=1, facecolor='blue', alpha=0.5)

xp = np.linspace(-2.32,2.32, 50)

# Adicionando uma distribuição normal
ya = norm.pdf(xp, mu, sigma)
plt.plot(xp, ya, 'r--')
plt.xlabel('Erros Padronizados')
plt.ylabel('Probabilidade')
plt.title(r'Histograma: $\mu=$'+str(mu)+' $\sigma=$'+str(sigma))

# Ajuste do espaçamento para evitar o recorte do rótulo
plt.subplots_adjust(left=0.15)
plt.rcParams['figure.figsize'] = (11,7)
plt.show()

nrm1=[ i for i in range(nc) if -sigma<rdp[i]<=sigma]
nrm2=[ i for i in range(nc) if -1.64*sigma<rdp[i]<=1.64*sigma]
nrm3=[ i for i in range(nc) if -1.96*sigma<rdp[i]<=1.96*sigma]

int1=len(nrm1)/(nc)
int2=len(nrm2)/(nc)
int3=len(nrm3)/(nc)

print ("A distribuição dos resíduos no intervalo de desvio padrão ]-1;+1[,]-1,64;+1,64[,]-1,96;+1,96[ são, respectivamente",int1*100,'%,',int2*100,'%,',int3*100,'% sendo o intervalo de Distribuição Normal sugerido (66 à 75% - 85 à 95% - 95 à 100%)')

if 0.66<=int1<=0.75 and 0.85<=int2<=0.95 and 0.95<=int3<=1:
 print ('Normalidade conforme a NBR-14.653-2 (A.2.1.2) está aderente')
else:
 print ('');

df3=var_dict[modesc].corr()

def color_red80p(val):
    color = 'red' if abs(val) > 0.8 else 'black'
    return 'color: %s' % color

print ('\033[91m','\033[1m')
print ("ANALISE DE CORRELAÇÃO ENTRE VARIÁVEIS ISOLADAS")

print ('\033[1m')
s=df3.style.applymap(color_red80p) #Analisar correlações maiores que 80%
display(s)

df3i=np.linalg.inv(df3)
print ('\033[91m','\033[1m')
print ("ANALISE DE CORRELAÇÃO PARCIAIS COM INFLUÊNCIA")

print ('\033[1m')

corp=[]
for k in range(nl):
    for i in range(nl):
        corp.append(-(df3i[i][k])/math.sqrt((df3i[i][i])*(df3i[k][k])));
                    
corp = np.reshape(corp , (nl, nl))

corp=pd.DataFrame(corp)

corp.columns=df3.columns
corp.index=corp.columns

sci=corp.style.applymap(color_red80p) #Analisar correlações maiores que 80%
display(sci)


print ('\033[91m','\033[1m')
print ("ANALISE DAS VARIÁVEIS MAIS RELEVANTES PARA O MODELO SEGUNDO A ANÁLISE DAS COMPONENTES PRINCIPAIS - PCA")


df6=df2.corr()
ev1,ev2=np.linalg.eig((df6.iloc[:-1,:-1])) #Autovalores,Autovetores


normaliz_df2=(df2-df2.mean())/df2.std()

print ("O autovalores que representam a variancia são:",ev1)

ev1p=ev1/sum(ev1)
print ("O autovalores que representam a variancia em porcentagem são:",ev1p)

sumav=0
ndpr=0
for k in range(nl-1):
    if sumav<0.8:
        sumav=sumav+ev1p[k]
        ndpr=k+1;

escor=np.matmul(normaliz_df2.iloc[:,:-1],ev2)

print ("Matriz dos escores dos dados nas direções principais")
escor=pd.DataFrame(escor)

escor.columns=[ 'CP'+str(i) for i in range(nl-1)]
display(escor)

corlvdp=[]
for k in range(nl-1):
    for i in range(nl-1):
        corlvdp.append((ev2[k][i])*sqrt(ev1[i]));
corlvdp = np.reshape(corlvdp , (nl-1, nl-1))


corlvdp=pd.DataFrame(corlvdp)

corlvdp.columns=escor.columns
corlvdp.index=df3.columns[:nl-1]

def color_blue80p(val):
    color = 'blue' if abs(val) > 0.8 else 'black'
    return 'color: %s' % color

print ('\033[1m')
print ("A correlação das variáveis com as direções principais")
sdp=corlvdp.style.applymap(color_blue80p) #Analisar correlações maiores que 80% nas direções principais
display(sdp)


varel=np.where(abs(corlvdp.iloc[:,:ndpr])>=0.8)

print ('\033[91m','\033[1m')
print("Apenas as variáveis:")
for k in varel[0]:
    print(df3.columns[:nl-1][k]);
print("Tem relevância e boa correlação nas direções principais")

In [15]:
type(escor)

pandas.core.frame.DataFrame

In [16]:
nc

In [17]:
nl

In [18]:
modf[91]

In [19]:
modf[91][2]